In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import models, layers, activations, initializers, losses, optimizers, metrics
from keras.models import Sequential, Model
from keras.layers import Dense, Input, LeakyReLU, Dropout, BatchNormalization
from keras.layers.merge import concatenate
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [61]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    length = len(sequence)
    col_length = len(sequence[0])
    for i in range(length):
        end_ix = i + n_steps
        if end_ix > length-1:
            break
        seq_x, seq_y = sequence[i:end_ix, :], sequence[end_ix, col_length-1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [62]:
kospi = pd.read_csv('data/kospi200.csv', encoding='euc-kr')

kospi = kospi.iloc[:,1:].copy()

kospi = pd.concat([kospi.iloc[:,:3], kospi.iloc[:,4], kospi.iloc[:,3]], axis=1)

kospi['거래량'] = kospi['거래량'].apply(lambda x: x.replace(",","",-1))

for i in kospi.columns:
    kospi[i] = kospi[i].astype(float)

kospi = np.array(kospi)

In [63]:
data = pd.read_csv('data/samsung.csv', encoding='euc-kr')

data = data.iloc[:,1:].copy()

data = pd.concat([data.iloc[:,:3], data.iloc[:,4], data.iloc[:,3]], axis=1)

for i in data.columns:
    data[i] = data[i].apply(lambda x: x.replace(",","",-1))

for i in data.columns:
    data[i] = data[i].astype(int)

data = np.array(data)

In [64]:
n_steps = 5
samsung_x, samsung_y = split_sequence(data, n_steps)
kospi_x, _ = split_sequence(kospi, n_steps)

In [65]:
samsung_x = samsung_x.reshape((samsung_x.shape[0], -1))
scaler = StandardScaler()
scaler.fit(samsung_x)
samsung_x = scaler.transform(samsung_x)
print(samsung_x.shape)
kospi_x = kospi_x.reshape((kospi_x.shape[0], -1))
scaler = StandardScaler()
scaler.fit(kospi_x)
kospi_x = scaler.transform(kospi_x)
print(kospi_x.shape)

(421, 25)
(421, 25)


In [66]:
final_predict_samsung, final_predict_kospi = samsung_x[-1:,:], kospi_x[-1:,:]
from sklearn.model_selection import train_test_split

samsung_x_train, samsung_x_test, kospi_x_train, kospi_x_test, y_train, y_test = train_test_split(
    samsung_x, kospi_x, samsung_y, train_size=0.9, random_state=66, shuffle=True
)

In [67]:
inp1 = Input(shape=(samsung_x.shape[1],))
x = Dense(128, activation='relu', kernel_initializer='he_normal')(inp1)
x = Dropout(0.25)(x)
x = Dense(64, activation='relu', kernel_initializer='he_normal')(x)
x = Dropout(0.25)(x)
out1 = Dense(32, activation="relu", kernel_initializer='he_normal')(x)

inp2 = Input(shape=(kospi_x.shape[1],))
x1 = Dense(128, activation='relu', kernel_initializer='he_normal')(inp2)
x = Dropout(0.25)(x)
x1 = Dense(64, activation='relu', kernel_initializer='he_normal')(x1)
x = Dropout(0.25)(x)
out2 = Dense(32, activation='relu', kernel_initializer='he_normal')(x1)

merge = concatenate([out1, out2])
merge = Dense(32, activation='relu', kernel_initializer='he_normal')(merge)
x = Dropout(0.25)(x)
merge = Dense(16, activation='relu', kernel_initializer='he_normal')(merge)
x = Dropout(0.25)(x)
output = Dense(1, activation='linear')(merge)

model = Model(inputs=[inp1, inp2], outputs=output)

In [68]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='loss', patience=50, mode='auto')
model.compile(loss='mse', optimizer='adam', metrics=['acc'])
model.fit([samsung_x_train, kospi_x_train], y_train, epochs=2000,
         validation_split=0.1, callbacks=[early_stopping])

Train on 340 samples, validate on 38 samples
Epoch 1/2000
340/340 [==============================] - 1s 4ms/step - loss: 2194528582.7765 - acc: 0.0000e+00 - val_loss: 2135281300.2105 - val_acc: 0.0000e+00
Epoch 2/2000
340/340 [==============================] - 0s 59us/step - loss: 2193806447.4353 - acc: 0.0000e+00 - val_loss: 2134037760.0000 - val_acc: 0.0000e+00
Epoch 3/2000
340/340 [==============================] - 0s 59us/step - loss: 2191844906.1647 - acc: 0.0000e+00 - val_loss: 2130484614.7368 - val_acc: 0.0000e+00
Epoch 4/2000
340/340 [==============================] - 0s 59us/step - loss: 2186218014.1176 - acc: 0.0000e+00 - val_loss: 2120474974.3158 - val_acc: 0.0000e+00
Epoch 5/2000
340/340 [==============================] - 0s 64us/step - loss: 2170097766.4000 - acc: 0.0000e+00 - val_loss: 2092454750.3158 - val_acc: 0.0000e+00
Epoch 6/2000
340/340 [==============================] - 0s 64us/step - loss: 2128248881.6941 - acc: 0.0000e+00 - val_loss: 2018364442.9474 - val_acc: 0

Epoch 52/2000
340/340 [==============================] - 0s 62us/step - loss: 61990119.5294 - acc: 0.0000e+00 - val_loss: 41465303.7895 - val_acc: 0.0000e+00
Epoch 53/2000
340/340 [==============================] - 0s 61us/step - loss: 57637783.4824 - acc: 0.0000e+00 - val_loss: 39571197.3684 - val_acc: 0.0000e+00
Epoch 54/2000
340/340 [==============================] - 0s 53us/step - loss: 58695338.1176 - acc: 0.0000e+00 - val_loss: 36555644.7368 - val_acc: 0.0000e+00
Epoch 55/2000
340/340 [==============================] - 0s 54us/step - loss: 53187808.2824 - acc: 0.0000e+00 - val_loss: 34724733.7895 - val_acc: 0.0000e+00
Epoch 56/2000
340/340 [==============================] - 0s 56us/step - loss: 52127386.4471 - acc: 0.0000e+00 - val_loss: 32076882.5263 - val_acc: 0.0000e+00
Epoch 57/2000
340/340 [==============================] - 0s 56us/step - loss: 51170562.8706 - acc: 0.0000e+00 - val_loss: 30807006.2105 - val_acc: 0.0000e+00
Epoch 58/2000
340/340 [=============================

Epoch 104/2000
340/340 [==============================] - 0s 58us/step - loss: 9666705.5941 - acc: 0.0000e+00 - val_loss: 4680647.8158 - val_acc: 0.0000e+00
Epoch 105/2000
340/340 [==============================] - 0s 55us/step - loss: 10661363.7647 - acc: 0.0000e+00 - val_loss: 4462469.6842 - val_acc: 0.0000e+00
Epoch 106/2000
340/340 [==============================] - 0s 55us/step - loss: 10595258.8647 - acc: 0.0000e+00 - val_loss: 4410704.0789 - val_acc: 0.0000e+00
Epoch 107/2000
340/340 [==============================] - 0s 53us/step - loss: 10466641.2176 - acc: 0.0000e+00 - val_loss: 4307771.6579 - val_acc: 0.0000e+00
Epoch 108/2000
340/340 [==============================] - 0s 66us/step - loss: 9467190.3294 - acc: 0.0000e+00 - val_loss: 4112629.7632 - val_acc: 0.0000e+00
Epoch 109/2000
340/340 [==============================] - 0s 53us/step - loss: 8593543.4000 - acc: 0.0000e+00 - val_loss: 3977704.2105 - val_acc: 0.0000e+00
Epoch 110/2000
340/340 [==============================]

340/340 [==============================] - 0s 62us/step - loss: 3303878.4471 - acc: 0.0029 - val_loss: 2097704.3816 - val_acc: 0.0000e+00
Epoch 157/2000
340/340 [==============================] - 0s 53us/step - loss: 3465039.4250 - acc: 0.0000e+00 - val_loss: 1985902.9474 - val_acc: 0.0000e+00
Epoch 158/2000
340/340 [==============================] - 0s 59us/step - loss: 3100454.4824 - acc: 0.0000e+00 - val_loss: 1989108.9342 - val_acc: 0.0000e+00
Epoch 159/2000
340/340 [==============================] - 0s 59us/step - loss: 2679094.9588 - acc: 0.0000e+00 - val_loss: 2055335.5921 - val_acc: 0.0000e+00
Epoch 160/2000
340/340 [==============================] - 0s 56us/step - loss: 2777404.0294 - acc: 0.0000e+00 - val_loss: 1815074.2500 - val_acc: 0.0000e+00
Epoch 161/2000
340/340 [==============================] - 0s 56us/step - loss: 2792667.8765 - acc: 0.0000e+00 - val_loss: 1854815.9605 - val_acc: 0.0000e+00
Epoch 162/2000
340/340 [==============================] - 0s 68us/step - loss

340/340 [==============================] - 0s 50us/step - loss: 1422836.6147 - acc: 0.0000e+00 - val_loss: 1556708.2763 - val_acc: 0.0000e+00
Epoch 209/2000
340/340 [==============================] - 0s 59us/step - loss: 1676103.8824 - acc: 0.0000e+00 - val_loss: 1528375.5263 - val_acc: 0.0000e+00
Epoch 210/2000
340/340 [==============================] - 0s 61us/step - loss: 1576208.1500 - acc: 0.0029 - val_loss: 1677593.3026 - val_acc: 0.0000e+00
Epoch 211/2000
340/340 [==============================] - 0s 60us/step - loss: 1608946.2426 - acc: 0.0000e+00 - val_loss: 1505701.5263 - val_acc: 0.0000e+00
Epoch 212/2000
340/340 [==============================] - 0s 62us/step - loss: 1792571.8147 - acc: 0.0000e+00 - val_loss: 1495282.9605 - val_acc: 0.0000e+00
Epoch 213/2000
340/340 [==============================] - 0s 64us/step - loss: 1708235.2882 - acc: 0.0000e+00 - val_loss: 1705059.6974 - val_acc: 0.0000e+00
Epoch 214/2000
340/340 [==============================] - 0s 59us/step - loss

...

340/340 [==============================] - 0s 59us/step - loss: 1000311.0956 - acc: 0.0000e+00 - val_loss: 1527386.7500 - val_acc: 0.0000e+00
Epoch 313/2000
340/340 [==============================] - 0s 61us/step - loss: 1016060.8265 - acc: 0.0000e+00 - val_loss: 1519393.8816 - val_acc: 0.0000e+00
Epoch 314/2000
340/340 [==============================] - 0s 57us/step - loss: 927777.6029 - acc: 0.0000e+00 - val_loss: 1625704.7500 - val_acc: 0.0000e+00
Epoch 315/2000
340/340 [==============================] - 0s 55us/step - loss: 952248.7882 - acc: 0.0029 - val_loss: 1350855.9079 - val_acc: 0.0000e+00
Epoch 316/2000
340/340 [==============================] - 0s 60us/step - loss: 767708.4485 - acc: 0.0000e+00 - val_loss: 1445339.1908 - val_acc: 0.0000e+00
Epoch 317/2000
340/340 [==============================] - 0s 60us/step - loss: 1101494.3581 - acc: 0.0000e+00 - val_loss: 1314906.6974 - val_acc: 0.0000e+00
Epoch 318/2000
340/340 [==============================] - 0s 62us/step - loss: 7

Epoch 365/2000
340/340 [==============================] - 0s 60us/step - loss: 839726.4007 - acc: 0.0000e+00 - val_loss: 1342127.0132 - val_acc: 0.0000e+00
Epoch 366/2000
340/340 [==============================] - 0s 63us/step - loss: 808678.6176 - acc: 0.0000e+00 - val_loss: 1508926.4145 - val_acc: 0.0000e+00
Epoch 367/2000
340/340 [==============================] - 0s 59us/step - loss: 789335.4875 - acc: 0.0000e+00 - val_loss: 1565784.4803 - val_acc: 0.0000e+00
Epoch 368/2000
340/340 [==============================] - 0s 59us/step - loss: 819048.2897 - acc: 0.0000e+00 - val_loss: 1360056.8355 - val_acc: 0.0000e+00
Epoch 369/2000
340/340 [==============================] - 0s 58us/step - loss: 841796.3460 - acc: 0.0029 - val_loss: 1275022.9605 - val_acc: 0.0000e+00
Epoch 370/2000
340/340 [==============================] - 0s 53us/step - loss: 721483.1309 - acc: 0.0000e+00 - val_loss: 1400636.4539 - val_acc: 0.0000e+00
Epoch 371/2000
340/340 [==============================] - 0s 59us/st

340/340 [==============================] - 0s 63us/step - loss: 836405.2515 - acc: 0.0000e+00 - val_loss: 1561456.4803 - val_acc: 0.0000e+00
Epoch 418/2000
340/340 [==============================] - 0s 60us/step - loss: 751637.5272 - acc: 0.0029 - val_loss: 1386620.0592 - val_acc: 0.0000e+00
Epoch 419/2000
340/340 [==============================] - 0s 58us/step - loss: 722109.6471 - acc: 0.0000e+00 - val_loss: 1176226.3684 - val_acc: 0.0000e+00
Epoch 420/2000
340/340 [==============================] - 0s 61us/step - loss: 700972.8691 - acc: 0.0000e+00 - val_loss: 1163848.9211 - val_acc: 0.0000e+00
Epoch 421/2000
340/340 [==============================] - 0s 53us/step - loss: 645531.6257 - acc: 0.0000e+00 - val_loss: 1249404.2895 - val_acc: 0.0000e+00
Epoch 422/2000
340/340 [==============================] - 0s 56us/step - loss: 685875.0820 - acc: 0.0000e+00 - val_loss: 1346268.0987 - val_acc: 0.0000e+00
Epoch 423/2000
340/340 [==============================] - 0s 63us/step - loss: 6756

Epoch 470/2000
340/340 [==============================] - 0s 62us/step - loss: 725338.8316 - acc: 0.0029 - val_loss: 1052862.2961 - val_acc: 0.0000e+00
Epoch 471/2000
340/340 [==============================] - 0s 67us/step - loss: 595112.3502 - acc: 0.0000e+00 - val_loss: 1369870.4474 - val_acc: 0.0000e+00
Epoch 472/2000
340/340 [==============================] - 0s 61us/step - loss: 576195.5250 - acc: 0.0029 - val_loss: 1390817.1513 - val_acc: 0.0000e+00
Epoch 473/2000
340/340 [==============================] - 0s 56us/step - loss: 850164.5640 - acc: 0.0000e+00 - val_loss: 1353394.9671 - val_acc: 0.0000e+00
Epoch 474/2000
340/340 [==============================] - 0s 56us/step - loss: 806666.2320 - acc: 0.0029 - val_loss: 1226093.6842 - val_acc: 0.0000e+00
Epoch 475/2000
340/340 [==============================] - 0s 56us/step - loss: 590902.6706 - acc: 0.0029 - val_loss: 1025116.8158 - val_acc: 0.0000e+00
Epoch 476/2000
340/340 [==============================] - 0s 53us/step - loss: 6

Epoch 523/2000
340/340 [==============================] - 0s 56us/step - loss: 573069.6654 - acc: 0.0000e+00 - val_loss: 1137051.3421 - val_acc: 0.0000e+00
Epoch 524/2000
340/340 [==============================] - 0s 62us/step - loss: 551351.6294 - acc: 0.0000e+00 - val_loss: 1123219.6908 - val_acc: 0.0000e+00
Epoch 525/2000
340/340 [==============================] - 0s 59us/step - loss: 563437.4838 - acc: 0.0000e+00 - val_loss: 1424947.8618 - val_acc: 0.0000e+00
Epoch 526/2000
340/340 [==============================] - 0s 61us/step - loss: 588581.0603 - acc: 0.0029 - val_loss: 1099105.8553 - val_acc: 0.0000e+00
Epoch 527/2000
340/340 [==============================] - 0s 55us/step - loss: 497177.0449 - acc: 0.0000e+00 - val_loss: 931137.1382 - val_acc: 0.0000e+00
Epoch 528/2000
340/340 [==============================] - 0s 59us/step - loss: 540590.5496 - acc: 0.0000e+00 - val_loss: 986080.1316 - val_acc: 0.0000e+00
Epoch 529/2000
340/340 [==============================] - 0s 62us/step

In [69]:
y_predict = model.predict([samsung_x_test, kospi_x_test])

In [70]:
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict): # 실제 정답값, 모델을 통한 예측값 
    return np.sqrt(mean_squared_error(y_test, y_predict))
print('RMSE :', RMSE(y_test, y_predict))

RMSE : 1245.6429225796473


In [71]:
print(model.predict([final_predict_samsung, final_predict_kospi]))

[[51358.117]]
